# Burbuja API Tutorial

The need may arise to use Burbuja from within a Python script (for a workflow, for instance). Hence why an API is available for using Burbuja. Complete the steps of this notebook to see examples of how to use Burbuja's API.

NOTE: You will need to install Burbuja and its dependencies, as well as NGLView:

```bash
pip install nglview
```

In [1]:
# paths - modify as needed
hsp90_structure_path = "~/Burbuja/Burbuja/tests/data/hsp90.pdb"
tryp_ben_dcd_path = "~/Burbuja/Burbuja/tests/data/tb_traj.dcd"
tryp_ben_prmtop_path = "~/Burbuja/Burbuja/tests/data/tryp_ben.prmtop"

In [2]:
# Imports and other preliminaries
import os
import time
import mdtraj
import nglview
import Burbuja.burbuja as burbuja
hsp90_structure_path = os.path.expanduser(hsp90_structure_path)
tryp_ben_dcd_path = os.path.expanduser(tryp_ben_dcd_path)
tryp_ben_prmtop_path = os.path.expanduser(tryp_ben_prmtop_path)

## 1. Simple Bubble Detection
First, we will use the API to simply detect whether a bubble exists in a HSP90 solvated structure.

In [3]:
hsp90_contains_bubble = burbuja.has_bubble(hsp90_structure_path)
print("HSP90 structure contains bubble:", hsp90_contains_bubble)

HSP90 structure contains bubble: False


No bubble is found in this structure.

## 2. Bubble Detection on a Trajectory

Next, we will see how to load a MDtraj trajectory object and run it through Burbuja with some alternative settings, like GPU acceleration.

In [4]:
traj_structure = mdtraj.load(tryp_ben_dcd_path, top=tryp_ben_prmtop_path)
traj_contains_bubble = burbuja.has_bubble(traj_structure)
print("Trajectory contains bubble:", traj_contains_bubble)

Trajectory contains bubble: True


Aha! We have found a bubble. Let's see where and how big...

The "dx_filename_base" argument defines the base name for the DX files, and indicates to Burbuja that we want to write the bubbles to DX.

In [5]:
dx_filename_base = "traj_bubble"
traj_contains_bubble = burbuja.has_bubble(traj_structure, dx_filename_base=dx_filename_base)

Bubble detected with volume: 2.214 nm^3. Frame: 0 Bubble: 0. Bubble volume map file: traj_bubble_frame_0_bubble_0.dx
Bubble detected with volume: 0.616 nm^3. Frame: 0 Bubble: 1. Bubble volume map file: traj_bubble_frame_0_bubble_1.dx
Bubble detected with volume: 7.695 nm^3. Frame: 0 Bubble: 2. Bubble volume map file: traj_bubble_frame_0_bubble_2.dx
Bubble detected with volume: 1.939 nm^3. Frame: 1 Bubble: 0. Bubble volume map file: traj_bubble_frame_1_bubble_0.dx
Bubble detected with volume: 0.423 nm^3. Frame: 1 Bubble: 1. Bubble volume map file: traj_bubble_frame_1_bubble_1.dx
Bubble detected with volume: 6.507 nm^3. Frame: 1 Bubble: 2. Bubble volume map file: traj_bubble_frame_1_bubble_2.dx
Bubble detected with volume: 1.094 nm^3. Frame: 2 Bubble: 0. Bubble volume map file: traj_bubble_frame_2_bubble_0.dx
Bubble detected with volume: 4.477 nm^3. Frame: 2 Bubble: 1. Bubble volume map file: traj_bubble_frame_2_bubble_1.dx
Bubble detected with volume: 0.118 nm^3. Frame: 2 Bubble: 2. Bub

It looks like Burbuja wrote 6 frames. Let's load them and visualize the resulting bubbles.

In [6]:
view_list = []
# Must be reloaded for visualization because Burbuja changes the MDTraj object
new_traj_structure = mdtraj.load(tryp_ben_dcd_path, top=tryp_ben_prmtop_path)
for i in range(6):
    traj_structure_this_frame = new_traj_structure[i]
    view = nglview.show_mdtraj(traj_structure_this_frame)
    for j in range(3):
        dx_filename = f"{dx_filename_base}_frame_{i}_bubble_{j}.dx"
        view.add_component(dx_filename, ext='dx')
    view.clear_representations()
    view.add_cartoon("protein")
    view.add_licorice("water")
    view.component_1.add_surface(opacity=0.25, wireframe=False, color="red", isolevel=0.5, isolevelType="value")
    view.component_2.add_surface(opacity=0.25, wireframe=False, color="red", isolevel=0.5, isolevelType="value")
    view.component_3.add_surface(opacity=0.25, wireframe=False, color="red", isolevel=0.5, isolevelType="value")
    view_list.append(view)

for i, view in enumerate(view_list):
    print(f"Iteration: {i}")
    display(view)

Iteration: 0


NGLWidget()

Iteration: 1


NGLWidget()

Iteration: 2


...

NGLWidget()

Iteration: 4


NGLWidget()

Iteration: 5


NGLWidget()

You should be able to see the bubble locations highlighted in red.

If we wanted the bubble detection to go faster, we could use GPU acceleration with CuPy:

In [7]:
dx_filename_base = "traj_bubble"
time_start = time.time()
traj_contains_bubble = burbuja.has_bubble(traj_structure, dx_filename_base=dx_filename_base, use_cupy=True)
time_end = time.time()
elapsed_time = time_end - time_start
print(f"Bubble detection completed in {elapsed_time:.2f} seconds.")

Bubble detected with volume: 2.214 nm^3. Frame: 0 Bubble: 0. Bubble volume map file: traj_bubble_frame_0_bubble_0.dx
Bubble detected with volume: 0.616 nm^3. Frame: 0 Bubble: 1. Bubble volume map file: traj_bubble_frame_0_bubble_1.dx
Bubble detected with volume: 7.695 nm^3. Frame: 0 Bubble: 2. Bubble volume map file: traj_bubble_frame_0_bubble_2.dx
Bubble detected with volume: 1.939 nm^3. Frame: 1 Bubble: 0. Bubble volume map file: traj_bubble_frame_1_bubble_0.dx
Bubble detected with volume: 0.423 nm^3. Frame: 1 Bubble: 1. Bubble volume map file: traj_bubble_frame_1_bubble_1.dx
Bubble detected with volume: 6.507 nm^3. Frame: 1 Bubble: 2. Bubble volume map file: traj_bubble_frame_1_bubble_2.dx
Bubble detected with volume: 1.094 nm^3. Frame: 2 Bubble: 0. Bubble volume map file: traj_bubble_frame_2_bubble_0.dx
Bubble detected with volume: 4.477 nm^3. Frame: 2 Bubble: 1. Bubble volume map file: traj_bubble_frame_2_bubble_1.dx
Bubble detected with volume: 0.118 nm^3. Frame: 2 Bubble: 2. Bub

We may also choose to change the default grid resolution with the 'grid_resolution' argument.

In [ ]:
traj_contains_bubble = burbuja.has_bubble(traj_structure, grid_resolution=0.05)

Incidentally, the '-r' argument to the command line tool is the same as the 'grid_resolution' argument in the API.

## 3. Accessing Bubble Objects

If you want more low-level access to information Burbuja obtains about structures, the `burbuja()` function returns a list of `Bubble` objects that have additional attributes and methods for analysis.

In [13]:
from Burbuja.modules.base import DEFAULT_MINIMUM_BUBBLE_VOLUME
bubbles = burbuja.burbuja(traj_structure)
for i, bubble in enumerate(bubbles):
    if bubble.total_bubble_volume > DEFAULT_MINIMUM_BUBBLE_VOLUME:
        print(f"Frame: {i}")
        print(f"Bubble detected with volume: {bubble.total_bubble_volume:.3f} nm^3.")
        print(f"System volume: {bubble.total_system_volume:.3f} nm^3.")
        bubble_percentage = 100.0 * (bubble.total_bubble_volume / bubble.total_system_volume)
        print(f"Bubbles occupy {bubble_percentage:.3f} % of the system, by volume.")
        

Frame: 0
Bubble detected with volume: 10.716 nm^3.
System volume: 198.630 nm^3.
Bubbles occupy 5.395 % of the system, by volume.
Frame: 1
Bubble detected with volume: 9.012 nm^3.
System volume: 195.641 nm^3.
Bubbles occupy 4.606 % of the system, by volume.
Frame: 2
Bubble detected with volume: 5.770 nm^3.
System volume: 192.567 nm^3.
Bubbles occupy 2.996 % of the system, by volume.
Frame: 3
Bubble detected with volume: 4.263 nm^3.
System volume: 188.649 nm^3.
Bubbles occupy 2.260 % of the system, by volume.
Frame: 4
Bubble detected with volume: 2.514 nm^3.
System volume: 185.268 nm^3.
Bubbles occupy 1.357 % of the system, by volume.
Frame: 5
Bubble detected with volume: 1.049 nm^3.
System volume: 182.873 nm^3.
Bubbles occupy 0.574 % of the system, by volume.
Frame: 6
Bubble detected with volume: 0.644 nm^3.
System volume: 180.735 nm^3.
Bubbles occupy 0.356 % of the system, by volume.
Frame: 7
Bubble detected with volume: 0.180 nm^3.
System volume: 178.727 nm^3.
Bubbles occupy 0.101 % o

## 4. Interactive Visualization

You may wish to interact with your molecules and bubble visualizations more comprehensively. One may use Py3Dmol for this.

In [11]:
import py3Dmol

iso_val = 0.5

burbuja.has_bubble(hsp90_structure_path, dx_filename_base="hsp90")
pdb_path = hsp90_structure_path
dx_path = "hsp90_frame_0.dx"

pdb_str = Path(pdb_path).read_text()
dx_str  = Path(dx_path).read_text()

# Create viewer
view = py3Dmol.view(width=900, height=650)
view.addModel(pdb_str, 'pdb')

# Protein look
view.setStyle({'cartoon': {'color': 'lightgrey'}})

# Waters (common residue names): small cyan spheres
oxygen_sel = {'resn': ['HOH','WAT','SOL','TIP3'], 'elem':'O'}
hydrogen_sel = {'resn': ['HOH','WAT','SOL','TIP3'], 'elem':'H'}
view.addStyle(oxygen_sel, {'sphere': {'scale': 0.25, 'color': 'red'}})
view.addStyle(hydrogen_sel, {'sphere': {'scale': 0.25, 'color': 'white'}})

# Add the isosurface from the DX volume (interactive & rotatable)
view.addVolumetricData(dx_str, 'dx', {
    'isoval': iso_val,
    'opacity': 0.5,
    'color': 'red'
})

# Nice touches
view.setBackgroundColor('white')
view.zoomTo()
view.show()

ModuleNotFoundError: No module named 'py3Dmol'

## 5. Tips & Notes

- Prefer **PDB input** for extremely large systems (fast custom reader; avoids some MDTraj overheads).
- `grid_resolution=0.1` (default) is 1 Å. Finer grids (e.g., `0.05`) reveal more detail but cost more time/memory.
- `use_cupy=True` enables **GPU acceleration** (if **CuPy + CUDA** are available).
- `dx_filename_base` writes **DX** maps for visualization and **bubble volumes per frame** for analysis.
- With **py3Dmol**, you can add more styles/selections (e.g., show only waters within 5 Å of the protein):